Marvin Acosta Arcpy Lab1

In [1]:
#This allow you to use arcpy system
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.5\\ArcToolbox\\Scripts')
import arcpy

Creating work enviroment

In [2]:
# This code allows you to create work environment 
arcpy.env.workspace = "‪U:\\geog458\\lab1\\"
arcpy.env.overwriteOutput = True

Adding a new field

In [2]:
# This code allows you to add a new field to shape file that will consist of the name of each counties
arcpy.AddField_management("U:\geog458\lab1\saep_bg10.shp", "cntyName", "TEXT") 
# Created a cursor to search in the table for the CountyName and FIPSCounty                        
myList = []
cursor = arcpy.da.SearchCursor("U:\geog458\lab1\WashingtonFIPS.dbf", ["CountyName", "FIPSCounty"])
# This code allows the cursor to loop in the shape file to find polygons that has the same FIPS code and putting
# it to the new column named cntyName
for row in cursor:
    cursoru = arcpy.da.UpdateCursor("U:\geog458\lab1\saep_bg10.shp", "cntyName", '"COUNTYFP10" = ' + "'%s'" %row[1])
    myList.append(row[0])
# To update the row

    for row2 in cursoru:
        row2[0] = row[0]
        cursoru.updateRow(row2)
        
# delete both cursors
del cursor
del cursoru


In [3]:
# Printing the result of the previous code 
print myList

[u'Adams', u'Asotin', u'Benton', u'Chelan', u'Clallam', u'Clark', u'Columbia', u'Cowlitz', u'Douglas', u'Ferry', u'Franklin', u'Garfield', u'Grant', u'Grays Harbor', u'Island', u'Jefferson', u'King', u'Kitsap', u'Kittitas', u'Klickitat', u'Lewis', u'Lincoln', u'Mason', u'Okanogan', u'Pacific', u'Pend Oreille', u'Pierce', u'San Juan', u'Skagit', u'Skamania', u'Snohomish', u'Spokane', u'Stevens', u'Thurston', u'Wahkiakum', u'Walla Walla', u'Whatcom', u'Whitman', u'Yakima']


Converting the original shape file to 39 different shape file (for each county)

In [14]:
# This code allows you to covert the original shape file to 39 different shape file for each county
for county in myList:
    arcpy.FeatureClassToFeatureClass_conversion("U:\geog458\lab1\saep_bg10.shp", 
                                                "U:\geog458\lab1\\",
                                               str(county),
                                               '"cntyName" = ' + "'%s'" %county)

In [15]:
from subprocess import call
import os

os.environ["GDAL_DATA"] = "C:\\OSGeo4W64\\share\\gdal"
os.environ["GDAL_DRIVER_PATH"] = "C:\\OSGeo4W64\\bin\\gdalplugins"
os.environ["PROJ_LIB"] = "C:\\OSGeo4W64\\share\\proj"
os.environ["PATH"] = "C:\\OSGeo4W64\\bin;"+os.environ["PATH"]+";C:\\OSGeo4W64\\apps\\msys\\bin;C:\\OSGeo4W64\\apps\\Python27\\Scripts"


Converting shape files into geojson files

In [17]:
# This code allows you to convert the shape files of each county you created into geojson files
for county in myList:
    call(['C:\\OSGeo4W64\\bin\\ogr2ogr',
      '-f','GeoJSON','-t_srs','WGS84',
      '-s_srs','EPSG:2927', # chaged the projection
      'U:\\geog458\\lab1\\geojson\\' + county + '.geojson',
      'U:\\geog458\\lab1\\' + county + '.shp'])

Adding all the population of each block that falls in the same county

In [5]:
# This code allow the curS to search on the origal shape file to look for POP2017 to add 
# all the population of each block that falls in the same county and put it into PopList 
PopList = []
for county in myList:
    curS = arcpy.da.SearchCursor("U:\geog458\lab1\saep_bg10.shp",["POP2017"],'"cntyName" = ' + "'%s'" %county)
    result = 0
    for row in curS:
        result = row[0] + result
    PopList.append(result)
# delete curS

del curS
# print the result of the code
print PopList

[19870.0, 22290.0, 193499.99999999997, 76829.99999999999, 74240.00000000001, 470999.9999999994, 4100.0, 105900.00000000001, 41420.0, 7740.0, 90330.00000000003, 2200.0, 95630.0, 72970.0, 82790.0, 31360.0, 2153700.0, 264300.00000000006, 44730.0, 21660.0, 77440.00000000003, 10700.0, 63190.00000000001, 42110.00000000001, 21249.999999999996, 13370.0, 859400.0000000003, 16510.0, 124100.00000000001, 11690.0, 789400.0000000005, 499800.0, 44510.0, 276899.9999999999, 4030.0, 61399.999999999985, 216300.00000000003, 48639.99999999999, 253000.0]


Print the 10 highest population 

In [7]:
# sorting the population by hioghest to lowest
PopList.sort(reverse=True)
# use for loop to figure out the 10 highest population counties and print the result
for i in range(10):
    print PopList[i]
    


2153700.0
859400.0
789400.0
499800.0
471000.0
276900.0
264300.0
253000.0
216300.0
193500.0


Probably it took me at least 4 days to work on this lab. At first I struggled and confused of where to begin. Also I worked with KIm Vo for this lab to helped me understand the process, function, and the code I used.